<a href="https://colab.research.google.com/github/Echo9k/job-posting-analysis/blob/master/HTML%20offline%20scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#@title Set up
#@markdown List of html filenames: **html_files**
import os
import re
import codecs
import requests

import pandas as pd

from bs4 import BeautifulSoup as bs

from tqdm import tqdm
from IPython.core.display import display, HTML
from requests.exceptions import HTTPError

!rm sample_data -r
directory = '/content/drive/My Drive/html_job_postings/'
html_files = next(os.walk(directory))[2]

rm: cannot remove 'sample_data': No such file or directory


# Downloading and Parsing Online Data

## Retrive data

In [7]:
#@title ### Testing connection

#@markdown For known sources and single HTML pages:

url = "https://api.github.com/" #@param ["https://api.github.com/", "http://www.python.org/", "https://techdevguide.withgoogle.com/"]
from urllib.request import urlopen

html_contents = urlopen(url).read()

display(requests.get('https://api.github.com'))
display(html_contents)

r = requests.get(url)
html_contents = r.text
for url in [url, url+'/invalid/']:
    try:
        response = requests.get(url)

        # If the response was successful, no Exception will be raised
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  # Python 3.6
    except Exception as err:
        print(f'Other error occurred: {err}')  # Python 3.6
    else:
        print('Success!')

<Response [200]>

b'{"current_user_url":"https://api.github.com/user","current_user_authorizations_html_url":"https://github.com/settings/connections/applications{/client_id}","authorizations_url":"https://api.github.com/authorizations","code_search_url":"https://api.github.com/search/code?q={query}{&page,per_page,sort,order}","commit_search_url":"https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}","emails_url":"https://api.github.com/user/emails","emojis_url":"https://api.github.com/emojis","events_url":"https://api.github.com/events","feeds_url":"https://api.github.com/feeds","followers_url":"https://api.github.com/user/followers","following_url":"https://api.github.com/user/following{/target}","gists_url":"https://api.github.com/gists{/gist_id}","hub_url":"https://api.github.com/hub","issue_search_url":"https://api.github.com/search/issues?q={query}{&page,per_page,sort,order}","issues_url":"https://api.github.com/issues","keys_url":"https://api.github.com/user/keys","label_sea

Success!
HTTP error occurred: 404 Client Error: Not Found for url: https://api.github.com//invalid/


## Getting a sense of the data

In [8]:
#@title read files to dir: html_dir
beautify = True #@param {type:"boolean"}
prettyfy = True #@param {type:"boolean"}

#@title Functions defined
def file_reader(file_name, subfolder='./'):
    with open(subfolder+file_name, "r") as f:
        return f.read()

render = lambda html_contents: display(HTML(html_contents))


html_dir = {}

for i in tqdm(html_files):
    key = i.replace('.html','').replace(' ', '_')
    value = file_reader(i, directory)
    if beautify:
        html_dir[key]=bs(value)
        if prettyfy:
            html_dir[key].prettify()
    else:
        html_dir[key]=value
#@markdown HTML files parsed to: html_dir
print(f"\n'html_dir':"
      f"\tkeys : {list(html_dir.keys())}")

100%|██████████| 1337/1337 [00:04<00:00, 313.72it/s]


'html_dir':	keys : ['77db13653acd38ec_fccid', '206ca14924ada590_fccid', '7a38c671a8ab6ccc_fccid', '8707c2ca919bba90_fccid', '1d938e69ee26dcab_fccid', '40d84746953486e1_fccid', '96c1b9e6f16c5d26_fccid', '4872a34a5dfdd332_fccid', '28c58e220f123695_fccid', '6723789ff541c8cf_fccid', '96cbe36bb86669af_fccid', '7a5337baf7235d76_fccid', '2bc3ec6e761963c0_fccid', 'b02aa271d2af7c26_fccid', '21091c4562d65586_fccid', '9260ae3565157b27_fccid', '428c84d73ad82e6d_fccid', '5b8e1f0e1f1f44d9_fccid', 'd7adfc46261ccdb3_fccid', 'f9ee01df580ba190_fccid', 'ba0fb38226f4d165_fccid', '659d53c7c386050f_fccid', 'b521d8275ab3eaae_fccid', '0bb8446d7e591e7c_fccid', 'f3b15a08db5ab069_fccid', 'f85ad23d857625d4_fccid', 'f8afd87cb9e11776_fccid', '74d64b882f79a0bd_fccid', '11513c8993b7cf2c_fccid', 'dd862b8711306a3b_fccid', '2419746e43442672_fccid', 'c5f6aa5216291758_fccid', 'd7a37a1c860ae69f_fccid', '9c561782a3045c1d_fccid', '174b6359679e8428_fccid', '76c76e140bffad11_fccid', 'bfdbc9e51b1e2f04_fccid', '4e10553b3a9bf947

In [9]:
#@markdown ##**Render**
#@markdown preview the pages with the function lambda
render = lambda html_contents: display(HTML(html_contents))
display_test = False #@param {type:"boolean"}
file_to_view = "00321a48d04fe754_fccid" #@param ["00321a48d04fe754_fccid"] {allow-input: true}

if display_test:
    html_contents = html_dir[file_to_view]
    render(html_contents)

In [13]:
#@title Extracting data to pandas.DataFrame

#@markdown *   Title
#@markdown *   Body
#@markdown *   Bullet points

number_of_samples = 5 #@param {type:"slider", min:0, max:10, step:1}

df = {key : [value.title.text, value.body.text, value.find_all('li')] for key, value in html_dir.items()}
df = pd.DataFrame(df, ['title', 'body', 'bullets']).transpose()
df.index.name = 'identifier'


df.sample(number_of_samples)

,title,body,bullets
identifier,,,
71a7e77480279ebf_fccid,"Data Science Intern - San Francisco, CA","Data Science Intern - San Francisco, CA\nJoin ...",[[[\nProven ability to assume a strong sense o...
4b53bf000ca326cd_fccid,"BA-Data Analystics & Visualization - Houston, ...","BA-Data Analystics & Visualization - Houston, ...",[[[Business Solutions & Application Developmen...
1c61a9e892e12f82_fccid,"Data & Applied Scientist - Redmond, WA","Data & Applied Scientist - Redmond, WA\nDo you...",[[\nSolve challenging problems at significant ...
b02aa271d2af7c26_fccid,"Data Analytics Intern - Los Angeles, CA 90015","Data Analytics Intern - Los Angeles, CA 90015\...","[[Manipulate, visualize, and analyze large dat..."
e39efeb75bc873c5_fccid,Cloud Comp Engr Associate - Alabama,Cloud Comp Engr Associate - Alabama\nJob Descr...,"[[[\nDesired:]], [[ITIL Foundation Certificati..."
